In [1]:
# Cell 1 — Install light dependencies (run once)
# (If you already have nltk or emoji, pip will skip/reuse them)
!pip install nltk emoji sentence-transformers xgboost scikit-learn imbalanced-learn --quiet

# Then in a new cell:
import re
import string
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import emoji

# Download NLTK data (run once)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# Cell 2 — Load your data
genz_df = pd.read_csv("Gen-Z - Sheet1.csv")
mill_df = pd.read_csv("old-ppl - Sheet1.csv")

print("Gen Z shape:", genz_df.shape)
print("Millennial shape:", mill_df.shape)
display(genz_df.head())
display(mill_df.head())


Gen Z shape: (299, 2)
Millennial shape: (299, 2)


,text,label
0,slay,0
1,lit,0
2,bet,0
3,fire,0
4,vibing,0


,text,label
0,awesome,0
1,chill,0
2,cool,0
3,dude,0
4,buddy,0


In [3]:
# Cell 3 — Slang map (start small, extend as needed)
# Add more entries from your dataset after inspecting rare tokens.
slang_map = {
    "slay": "awesome",
    "lit": "amazing",
    "bet": "okay",
    "fire": "great",
    "vibing": "enjoying",
    "cap": "lie",
    "no cap": "no_lie",
    "fr": "for_real",
    "sus": "suspicious",
    "delulu": "delusional",
    "ratio": "disagree",
    "tf": "what_the",
    "idk": "i_do_not_know",
    "ikr": "i_know_right",
    "brb": "be_right_back",
    # add tokens as you see them
}


In [4]:
# Cell 4 — Emoji map for common emoji -> text
# Extend as you find new emojis in your data.
emoji_map = {
    "😂": "laughing",
    "😭": "crying",
    "😡": "angry",
    "😠": "angry",
    "😊": "happy",
    "🙂": "smile",
    "😍": "love",
    "❤️": "love",
    "💀": "dead_funny",
    "🤡": "clown",
    "🙄": "annoyed",
    "😒": "unimpressed",
    "🔥": "hot",
    "✨": "sparkle",
    "💅": "sass",
    "🤝": "agreement",
    "👍": "thumbs_up",
    "👎": "thumbs_down",
    "🤬": "swearing",
    "😢": "sad",
    "😅": "nervous_laugh",
    # add more as you encounter them
}


In [5]:
# Cell 5 — Preprocessing functions
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def replace_emojis(text):
    # First try to use our emoji_map for quick replacements
    for em, meaning in emoji_map.items():
        if em in text:
            text = text.replace(em, " " + meaning + " ")
    # Then convert any remaining emoji unicode to text fallback using emoji.demojize
    # demojize outputs like :grinning_face: — we convert to words
    try:
        dem = emoji.demojize(text)
        # replace :smiling_face: -> smiling_face
        dem = re.sub(r':([a-zA-Z0-9_+-]+):', r' \1 ', dem)
        return dem
    except Exception:
        return text

def replace_slangs(text):
    # word-boundary replace for slang map; lowercased
    text = " " + text + " "
    for s, rep in slang_map.items():
        # handle multi-word slang like "no cap"
        pattern = r'\b' + re.escape(s.lower()) + r'\b'
        text = re.sub(pattern, " " + rep + " ", text, flags=re.IGNORECASE)
    return text

def clean_text(text, do_lemmatize=True, remove_stopwords=False):
    if not isinstance(text, str):
        text = str(text)
    text = text.strip()
    text = text.lower()
    # replace emojis
    text = replace_emojis(text)
    # replace slang tokens
    text = replace_slangs(text)
    # remove urls, mentions, hashtags (but keep hashtag words)
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'#', ' ', text)
    # remove punctuation except for underscore used in replacements
    text = re.sub(r'[{}]'.format(re.escape(string.punctuation.replace('_', ''))), ' ', text)
    # remove digits
    text = re.sub(r'\d+', ' ', text)
    # collapse whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = text.split()
    if remove_stopwords:
        tokens = [t for t in tokens if t not in stop_words]
    if do_lemmatize:
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)


In [6]:
# Cell 6 — Apply preprocessing and inspect
genz_df['clean_text'] = genz_df['text'].apply(lambda x: clean_text(x, do_lemmatize=True, remove_stopwords=False))
mill_df['clean_text'] = mill_df['text'].apply(lambda x: clean_text(x, do_lemmatize=True, remove_stopwords=False))

# Quick checks
print("Sample GenZ cleaned:")
display(genz_df[['text','clean_text']].head(10))
print("Sample Mill cleaned:")
display(mill_df[['text','clean_text']].head(10))

# Frequency of tokens — helpful to spot uncovered slang/emojis
def top_tokens(series, n=30):
    c = Counter()
    for s in series.dropna():
        c.update(s.split())
    return c.most_common(n)

print("Top GenZ tokens:", top_tokens(genz_df['clean_text'], 30))
print("Top Mill tokens:", top_tokens(mill_df['clean_text'], 30))


Sample GenZ cleaned:


,text,clean_text
0,slay,awesome
1,lit,amazing
2,bet,okay
3,fire,great
4,vibing,enjoying
5,chill,chill
6,aesthetic,aesthetic
7,wholesome,wholesome
8,valid,valid
9,iconic,iconic


Sample Mill cleaned:


,text,clean_text
0,awesome,awesome
1,chill,chill
2,cool,cool
3,dude,dude
4,buddy,buddy
5,amazing,amazing
6,great,great
7,lovely,lovely
8,happy,happy
9,peace,peace


Top GenZ tokens: [('yeah', 39), ('you', 36), ('oh', 29), ('is', 23), ('you’re', 20), ('the', 19), ('sure', 19), ('your', 17), ('wow', 17), ('a', 16), ('smirking_face', 15), ('just', 14), ('another', 14), ('love', 13), ('keep', 13), ('that', 12), ('i', 12), ('being', 12), ('grimacing_face', 12), ('this', 11), ('and', 11), ('relieved_face', 9), ('cry', 9), ('totally', 9), ('nervous_laugh', 9), ('energy', 8), ('to', 8), ('great', 7), ('chill', 7), ('vibe', 7)]
Top Mill tokens: [('you', 55), ('you’re', 31), ('a', 29), ('oh', 26), ('because', 26), ('laughing', 25), ('my', 24), ('your', 24), ('smirking_face', 23), ('love', 22), ('to', 22), ('annoyed', 22), ('wow', 20), ('another', 20), ('is', 19), ('grimacing_face', 19), ('the', 18), ('that', 17), ('i', 16), ('this', 16), ('like', 15), ('so', 14), ('how', 14), ('make', 13), ('for', 13), ('just', 13), ('—', 12), ('sure', 12), ('yes', 12), ('great', 11)]


In [7]:
# Cell 7 — Export cleaned datasets for training stage
genz_df.to_csv("genz_cleaned.csv", index=False)
mill_df.to_csv("mill_cleaned.csv", index=False)
print("Saved cleaned files to /mnt/data/")

Saved cleaned files to /mnt/data/


In [8]:
# Cell 8 — Imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

import joblib
import warnings
warnings.filterwarnings('ignore')


In [10]:
# Cell 9 — Load cleaned datasets
genz_df = pd.read_csv("genz_cleaned.csv")
mill_df = pd.read_csv("mill_cleaned.csv")

print("Gen-Z Data:", genz_df.shape)
print("Millennial Data:", mill_df.shape)


Gen-Z Data: (299, 3)
Millennial Data: (299, 3)


In [27]:
# ==============================================
# 🧩 Cell 10 — Generate sentence embeddings (MPNet + sentiment)
# Clean + silent version (no widget warnings)
# ==============================================
import warnings
warnings.filterwarnings("ignore")  # hide all warnings

from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import numpy as np

# disable tqdm progress bars globally
tqdm.disable = True

# better embedding model for sarcasm + tone awareness
model_embed = SentenceTransformer('paraphrase-mpnet-base-v2')

# sentiment model adds emotional cues (POSITIVE/NEGATIVE + confidence)
sentiment_analyzer = pipeline("sentiment-analysis")

def get_embeddings_with_sentiment(texts):
    # contextual sentence embeddings
    emb = model_embed.encode(texts, show_progress_bar=False)

    # sentiment direction (+1/-1) and strength
    sentiments = sentiment_analyzer(texts)
    sent_scores = np.array([[1 if s['label'] == 'POSITIVE' else -1, s['score']] for s in sentiments])

    # merge embeddings + sentiment for richer representation
    return np.concatenate((emb, sent_scores), axis=1)

print("Generating embeddings for GenZ and Millennials (please wait)...")

X_genz = get_embeddings_with_sentiment(genz_df['clean_text'].tolist())
y_genz = genz_df['label'].values

X_mill = get_embeddings_with_sentiment(mill_df['clean_text'].tolist())
y_mill = mill_df['label'].values

print("✅ Embeddings ready! (No widget errors, fully clean run)")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Generating embeddings for GenZ and Millennials (please wait)...
✅ Embeddings ready! (No widget errors, fully clean run)


In [28]:
# ==============================================
# 🧩 Cell 11 — Split into Train/Test
# ==============================================
from sklearn.model_selection import train_test_split
import numpy as np

print("Splitting GenZ & Millennial data...")

Xg_train, Xg_test, yg_train, yg_test = train_test_split(X_genz, y_genz, test_size=0.2, random_state=42)
Xm_train, Xm_test, ym_train, ym_test = train_test_split(X_mill, y_mill, test_size=0.2, random_state=42)

print("✅ Data split complete.")


Splitting GenZ & Millennial data...
✅ Data split complete.


In [29]:
# ==============================================
# 🧩 Cell 12 — Define Base Models
# ==============================================
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

print("Initializing base models...")

svm_clf = SVC(probability=True, kernel='linear', C=1, random_state=42)
rf_clf = RandomForestClassifier(n_estimators=150, max_depth=10, random_state=42)
xgb_clf = XGBClassifier(
    n_estimators=150, 
    max_depth=6, 
    learning_rate=0.1, 
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)

print("✅ Models initialized.")


Initializing base models...
✅ Models initialized.


In [30]:
# ==============================================
# 🧩 Cell 13 — Train Base + Meta (Gen-Z)
# ==============================================
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print("Training ensemble for Gen-Z...")

# train base models
svm_clf.fit(Xg_train, yg_train)
rf_clf.fit(Xg_train, yg_train)
xgb_clf.fit(Xg_train, yg_train)

# validation split
Xg_train_sub, Xg_val, yg_train_sub, yg_val = train_test_split(Xg_train, yg_train, test_size=0.2, random_state=42)

# meta features
p1 = svm_clf.predict_proba(Xg_val)[:, 1]
p2 = rf_clf.predict_proba(Xg_val)[:, 1]
p3 = xgb_clf.predict_proba(Xg_val)[:, 1]
X_meta_g = np.column_stack((p1, p2, p3))

meta_clf_g = LogisticRegression(max_iter=1000)
meta_clf_g.fit(X_meta_g, yg_val)

# evaluate
p1_test = svm_clf.predict_proba(Xg_test)[:, 1]
p2_test = rf_clf.predict_proba(Xg_test)[:, 1]
p3_test = xgb_clf.predict_proba(Xg_test)[:, 1]
X_meta_test_g = np.column_stack((p1_test, p2_test, p3_test))
pred_meta_g = meta_clf_g.predict(X_meta_test_g)

print(f"✅ Gen-Z Ensemble Accuracy: {accuracy_score(yg_test, pred_meta_g):.3f}")


Training ensemble for Gen-Z...
✅ Gen-Z Ensemble Accuracy: 0.900


In [31]:
# ==============================================
# 🧩 Cell 14 — Train Base + Meta (Millennials)
# ==============================================
print("Training ensemble for Millennials...")

# base models
svm_clf.fit(Xm_train, ym_train)
rf_clf.fit(Xm_train, ym_train)
xgb_clf.fit(Xm_train, ym_train)

# validation
Xm_train_sub, Xm_val, ym_train_sub, ym_val = train_test_split(Xm_train, ym_train, test_size=0.2, random_state=42)

p1 = svm_clf.predict_proba(Xm_val)[:, 1]
p2 = rf_clf.predict_proba(Xm_val)[:, 1]
p3 = xgb_clf.predict_proba(Xm_val)[:, 1]
X_meta_m = np.column_stack((p1, p2, p3))

meta_clf_m = LogisticRegression(max_iter=1000)
meta_clf_m.fit(X_meta_m, ym_val)

# evaluate
p1_test = svm_clf.predict_proba(Xm_test)[:, 1]
p2_test = rf_clf.predict_proba(Xm_test)[:, 1]
p3_test = xgb_clf.predict_proba(Xm_test)[:, 1]
X_meta_test_m = np.column_stack((p1_test, p2_test, p3_test))
pred_meta_m = meta_clf_m.predict(X_meta_test_m)

print(f"✅ Millennial Ensemble Accuracy: {accuracy_score(ym_test, pred_meta_m):.3f}")


Training ensemble for Millennials...
✅ Millennial Ensemble Accuracy: 0.967


In [32]:
# ==============================================
# 🧩 Cell 15 — Save Trained Models
# ==============================================
import joblib

print("Saving all trained models...")

# Gen-Z
joblib.dump(svm_clf, "svm_model_genz.pkl")
joblib.dump(rf_clf, "rf_model_genz.pkl")
joblib.dump(xgb_clf, "xgb_model_genz.pkl")
joblib.dump(meta_clf_g, "meta_model_genz.pkl")

# Millennials (retrain for clean independent set)
svm_mill = SVC(probability=True, kernel='linear', C=1, random_state=42)
rf_mill = RandomForestClassifier(n_estimators=150, max_depth=10, random_state=42)
xgb_mill = XGBClassifier(
    n_estimators=150, 
    max_depth=6, 
    learning_rate=0.1, 
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)

svm_mill.fit(Xm_train, ym_train)
rf_mill.fit(Xm_train, ym_train)
xgb_mill.fit(Xm_train, ym_train)

joblib.dump(svm_mill, "svm_model_mill.pkl")
joblib.dump(rf_mill, "rf_model_mill.pkl")
joblib.dump(xgb_mill, "xgb_model_mill.pkl")
joblib.dump(meta_clf_m, "meta_model_mill.pkl")

print("💾 All models saved successfully — clean, quiet, and ready for deployment.")


Saving all trained models...
💾 All models saved successfully — clean, quiet, and ready for deployment.


In [33]:
# ==============================================
# 🧩 Cell 16 — Emoji + Slang Normalization Layer
# ==============================================
import re
import emoji

print("Setting up emoji + slang normalization...")

emoji_map = {
    "😂": "laughing", "🤣": "laughing", "😭": "crying", "😡": "angry", "😠": "angry",
    "💀": "dead funny", "❤️": "love", "💔": "heartbroken", "😒": "annoyed", "🙄": "annoyed",
    "🔥": "fire", "💅": "sassy", "😎": "cool", "🤡": "clown", "🤬": "offensive", "😈": "evil",
    "🤔": "thinking", "😩": "tired", "😜": "playful", "😉": "flirty"
}

slang_map = {
    "bruh": "bro", "idk": "i dont know", "lmao": "laughing", "rofl": "laughing",
    "wtf": "what the hell", "smh": "disappointed", "omg": "oh my god", "af": "very",
    "lit": "amazing", "slay": "killing it", "lowkey": "a little", "highkey": "very much",
    "sus": "suspicious", "fr": "for real", "cap": "lie", "no cap": "truth",
    "salty": "bitter", "tea": "gossip", "flex": "show off"
}

def normalize_text(text: str) -> str:
    """Cleans emojis, slang & punctuation — returns normalized lowercase text."""
    # Convert emojis to words
    for emo, meaning in emoji_map.items():
        text = text.replace(emo, f" {meaning} ")
    text = emoji.demojize(text)

    # Expand slang
    for slang, expanded in slang_map.items():
        pattern = r"\b" + re.escape(slang) + r"\b"
        text = re.sub(pattern, expanded, text, flags=re.IGNORECASE)

    # Clean punctuation & extra spaces
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip().lower()
    return text

# quick sanity check
sample = "bruh 😂 that roast was lit 💀"
print("Example →", normalize_text(sample))
print("✅ Normalization layer ready.")


Setting up emoji + slang normalization...
Example → bro laughing that roast was amazing dead funny
✅ Normalization layer ready.


In [35]:
# ==============================================
# 🧩 Cell 17 — Unified Prediction Function
# ==============================================
import numpy as np
import joblib
from sentence_transformers import SentenceTransformer

print("Loading models & embedding pipeline...")

# load models
svm_genz = joblib.load("svm_model_genz.pkl")
rf_genz = joblib.load("rf_model_genz.pkl")
xgb_genz = joblib.load("xgb_model_genz.pkl")
meta_genz = joblib.load("meta_model_genz.pkl")

svm_mill = joblib.load("svm_model_mill.pkl")
rf_mill = joblib.load("rf_model_mill.pkl")
xgb_mill = joblib.load("xgb_model_mill.pkl")
meta_mill = joblib.load("meta_model_mill.pkl")

model_embed = SentenceTransformer('paraphrase-mpnet-base-v2')

print("✅ All models & embedding layers loaded successfully.\n")

def predict_text(text: str, mode: str = "genz") -> dict:
    """
    Predicts if text is offensive or non-offensive.
    mode: 'genz' or 'mill' (for Millennials)
    """
    # Step 1 — Normalize text
    clean_text = normalize_text(text)

    # Step 2 — Embed
    # Step 2 — Embed (same as training)
    base_emb = model_embed.encode([clean_text])
    sentiment = sentiment_analyzer([clean_text])[0]
    sent_dir = 1 if sentiment['label'] == 'POSITIVE' else -1
    sent_conf = sentiment['score']
    sent_feat = np.array([[sent_dir, sent_conf]])
    emb = np.concatenate((base_emb, sent_feat), axis=1)


    # Step 3 — Select correct ensemble
    if mode.lower() == "genz":
        p1 = svm_genz.predict_proba(emb)[:, 1]
        p2 = rf_genz.predict_proba(emb)[:, 1]
        p3 = xgb_genz.predict_proba(emb)[:, 1]
        X_meta = np.column_stack((p1, p2, p3))
        pred = meta_genz.predict(X_meta)[0]
        conf = meta_genz.predict_proba(X_meta)[0][pred]
    else:
        p1 = svm_mill.predict_proba(emb)[:, 1]
        p2 = rf_mill.predict_proba(emb)[:, 1]
        p3 = xgb_mill.predict_proba(emb)[:, 1]
        X_meta = np.column_stack((p1, p2, p3))
        pred = meta_mill.predict(X_meta)[0]
        conf = meta_mill.predict_proba(X_meta)[0][pred]

    label = "Cyberbullying / Offensive" if pred == 1 else "Non-offensive"
    return {"label": label, "confidence": round(float(conf) * 100, 2)}

# sanity test
sample1 = "bruh 😂 u so dumb 💀"
sample2 = "i love your outfit 💅"
print("Gen-Z:", predict_text(sample1, mode="genz"))
print("Millennials:", predict_text(sample2, mode="mill"))
print("\n✅ Unified prediction function working perfectly.")


Loading models & embedding pipeline...
✅ All models & embedding layers loaded successfully.

Gen-Z: {'label': 'Cyberbullying / Offensive', 'confidence': 90.2}
Millennials: {'label': 'Non-offensive', 'confidence': 89.52}

✅ Unified prediction function working perfectly.
